In [1]:
import pandas as pd
import openai
from dotenv import load_dotenv
load_dotenv()
import os
from langchain.prompts import load_prompt
from langchain.llms import OpenAIChat
import sys
pd.options.mode.chained_assignment = None
from langchain.callbacks import get_openai_callback
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader

from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    SystemMessage
)
import sys
sys.path.append('./modules/')
openai.api_key = os.getenv('OPENAI_KEY')
key = os.getenv('OPENAI_KEY')

from Generator import Generator
from Extractor import Extractor
from PromptPath import PromptPath
from GenExtract import GenExtract
from Property import Property
from NaturalLanguagePool import NaturalLanguagePool
from DynamicChatHistory import DynamicChatHistory
from Conversation import Conversation
from ConversationScript import ConversationScript
from ConversationScriptStage import ConversationScriptStage
from show_chat_log import show_chat_log
from is_question import is_question
from customer_profile import customer_profile

times_embedding_run = 0

#### App Development Notes:

What seems to be creating the greatest latency issues?
1. The memory, which was supposed to be a token saving/context management system but may be overkill in early stage parts of the conversation
2. The classifers that are needed to preare the prompt (the prompt itself seems to run pretty fast)

In [2]:
#load in the policies to the vectorstore
loaders = []
docs = []

#create a list with the names of the files to be loaded in the data/policies folder
for file in os.listdir('data/policies'):
    if file.endswith('.txt'):
        loader = TextLoader('data/policies/' + file)
        loaders.append(loader)

for loader in loaders:
    docs.extend(loader.load())

if times_embedding_run < 1:
    text_splitter = CharacterTextSplitter(chunk_size=250, chunk_overlap=0)
    documents = text_splitter.split_documents(docs)

    embeddings = OpenAIEmbeddings(openai_api_key=key)
    policies = Chroma.from_documents(documents, embeddings)
    times_embedding_run += 1

"Embeddings generated into policies variable"

Created a chunk of size 408, which is longer than the specified 250
Created a chunk of size 377, which is longer than the specified 250
Created a chunk of size 293, which is longer than the specified 250
Created a chunk of size 298, which is longer than the specified 250
Created a chunk of size 332, which is longer than the specified 250
Created a chunk of size 510, which is longer than the specified 250
Created a chunk of size 514, which is longer than the specified 250
Created a chunk of size 350, which is longer than the specified 250
Created a chunk of size 265, which is longer than the specified 250
Created a chunk of size 458, which is longer than the specified 250
Created a chunk of size 667, which is longer than the specified 250
Created a chunk of size 282, which is longer than the specified 250
Created a chunk of size 364, which is longer than the specified 250
Created a chunk of size 441, which is longer than the specified 250
Created a chunk of size 360, which is longer tha

Running Chroma using direct local API.
Using DuckDB in-memory for database. Data will be transient.


'Embeddings generated into policies variable'

In [3]:
targets = Property('targets', None, "Any target pests or rodents that the customer is having issues with", required=True, examples=['roaches', 'rats', 'mice'])

location_of_issue = Property('location_of_issue', None, "The location of the issue", required=True, examples=['kitchen', 'bathroom', 'bedroom', 'proactive'])

special_instructions = Property('special_instructions', None, "Any special instructions for the technician", required=False, examples=['please call before arriving', 'please leave a note on the door'])

square_footage = Property('square_footage', None, "The square footage of the property", required=False, examples=['1000', '2000', '3000'])

pest_id = Property('pest_id', None, "The id number of the pest", required=False, examples=['1', '2', '3'])

issues = NaturalLanguagePool([targets, location_of_issue, special_instructions, pest_id, square_footage], "The issues the customer is having", create_new_properties=True)

In [7]:
chat = ChatOpenAI(openai_api_key=key)

initial_prompt = """You are a helpful sales agent communicating with a prospect for Better Termite & Pest Control through a chat window. You should come off as cool, relaxed and informal, but helpful. A new lead has reached out through our site. I will provide you with goals based on the information you have collected so far. Try to avoid asking multiple questions in one response. Avoid overly formal words and phrases."""

history = DynamicChatHistory(chat, system_message=initial_prompt, additional_context="", initial_history=[SystemMessage(content="Hi! My name is George from Better Termite & Pest Control. How can I help you? :)")])

stage_one = ConversationScriptStage('Take lead of conversation', "Ask the customer's permission to ask questions to understand their issue. This goal is considered complete, once they have agreed to let you lead the conversation.", lambda: any(isinstance(message, AIMessage) for message in history.history))

stage_two = ConversationScriptStage('Get contact information', "Get the customer's contact information. This goal is considered complete, once you have their first name, last name, phone number, email, street address, city, state and zip code.", lambda: customer_profile.is_complete(), pool=customer_profile, show_missing=True, show_complete=True)

stage_three = ConversationScriptStage('Get customer issue', "Get the customer's issue. This goal is considered complete, once you have their issue, severity, location of issue, service type, special instructions and time constraints.", lambda: issues.is_complete(), pool=issues, show_missing=True, show_complete=True)

sales_script = ConversationScript([stage_one, stage_two, stage_three])

agent = Conversation(history, chat, sales_script, customer_profile, issues, policies, log_prompt=True)

In [8]:
agent.respond('Hi name is John Smith and I am interested in pest control')

*System*:  You are a helpful sales agent communicating with a prospect for Better Termite & Pest Control through a chat window. You should come off as cool, relaxed and informal, but helpful. A new lead has reached out through our site. I will provide you with goals based on the information you have collected so far. Try to avoid asking multiple questions in one response. Avoid overly formal words and phrases.
Conversation Summary So Far:
User: Hi name is John Smith and I am interested in pest control
 Current goal: Ask the customer's permission to ask questions to understand their issue. This goal is considered complete, once they have agreed to let you lead the conversation.Use the following pieces of context to answer the users question. If you don't know the answer, just say that you don't know, don't try to make up an answer.
At our company, we strive to provide prompt and efficient pest control services to all of our customers. To ensure that our scheduling process runs smoothly,

In [9]:
agent.respond('Yeah sure, thats fine')

*System*:  You are a helpful sales agent communicating with a prospect for Better Termite & Pest Control through a chat window. You should come off as cool, relaxed and informal, but helpful. A new lead has reached out through our site. I will provide you with goals based on the information you have collected so far. Try to avoid asking multiple questions in one response. Avoid overly formal words and phrases.
Conversation Summary So Far:
User: Hi name is John Smith and I am interested in pest control
Agent: Hey John, I'd be happy to assist you with that. Before we proceed, would you mind if I ask a few questions to better understand what pest issue you're facing?
User: Yeah sure, thats fine
 Current goal: Get the customer's contact information. This goal is considered complete, once you have their first name, last name, phone number, email, street address, city, state and zip code.
What we are missing: phone_number, email, street_address, city, state, zip
What we have so far:
first_na

In [10]:
agent.respond('Sure my name is George Schulz. My email is georgeschulz33@gmail.com, phone is 571-431-9531, and my address is 123 Elm Way, Gaithersburg, MD 20878')

*System*:  You are a helpful sales agent communicating with a prospect for Better Termite & Pest Control through a chat window. You should come off as cool, relaxed and informal, but helpful. A new lead has reached out through our site. I will provide you with goals based on the information you have collected so far. Try to avoid asking multiple questions in one response. Avoid overly formal words and phrases.
Conversation Summary So Far:
User: Hi name is John Smith and I am interested in pest control
Agent: Hey John, I'd be happy to assist you with that. Before we proceed, would you mind if I ask a few questions to better understand what pest issue you're facing?
User: Yeah sure, thats fine
Agent: Great, thanks for your cooperation! Can you please provide me with your street address, city, state, and zip code so that we can locate your property and better understand your situation? If you don't feel comfortable providing that information just yet, no worries. We can move forward with 

In [14]:
agent.respond("I've been seeing ants in my kitchen and some spiders around the house. Im just fed up!")

*System*:  You are a helpful sales agent communicating with a prospect for Better Termite & Pest Control through a chat window. You should come off as cool, relaxed and informal, but helpful. A new lead has reached out through our site. I will provide you with goals based on the information you have collected so far. Try to avoid asking multiple questions in one response. Avoid overly formal words and phrases.
Conversation Summary So Far:
User: Hi name is John Smith and I am interested in pest control
Agent: Hey John, I'd be happy to assist you with that. Before we proceed, would you mind if I ask a few questions to better understand what pest issue you're facing?
User: Yeah sure, thats fine
Agent: Great, thanks for your cooperation! Can you please provide me with your street address, city, state, and zip code so that we can locate your property and better understand your situation? If you don't feel comfortable providing that information just yet, no worries. We can move forward with 

In [15]:
agent.respond('The problem is pretty bad. I have seen them in my pantry and the DIY stuff isnt working. I would like to get rid of them ASAP. I do need to make sure the tech knows that I have kids and pets though!')

*System*:  You are a helpful sales agent communicating with a prospect for Better Termite & Pest Control through a chat window. You should come off as cool, relaxed and informal, but helpful. A new lead has reached out through our site. I will provide you with goals based on the information you have collected so far. Try to avoid asking multiple questions in one response. Avoid overly formal words and phrases.
Conversation Summary So Far:
User: Hi name is John Smith and I am interested in pest control
Agent: Hey John, I'd be happy to assist you with that. Before we proceed, would you mind if I ask a few questions to better understand what pest issue you're facing?
User: Yeah sure, thats fine
Agent: Great, thanks for your cooperation! Can you please provide me with your street address, city, state, and zip code so that we can locate your property and better understand your situation? If you don't feel comfortable providing that information just yet, no worries. We can move forward with 

In [10]:
agent.customer_profile.data

[first_name: George,
 last_name: Schulz,
 phone_number: 571-431-9531,
 email: georgeschulz33@gmail.com,
 street_address: 123 Elm Way,
 city: Gaithersburg,
 state: MD,
 zip: 20878]

In [16]:
show_chat_log(agent.history)

*Initialization Prompt*: Hi! My name is George from Better Termite & Pest Control. How can I help you? :)
------------------------------------
*User*: Hi name is John Smith and I am interested in pest control
------------------------------------
*Agent*: Hey John, I'd be happy to assist you with that. Before we proceed, would you mind if I ask a few questions to better understand what pest issue you're facing?
------------------------------------
*User*: Yeah sure, thats fine
------------------------------------
*Agent*: Great, thanks for your cooperation! Can you please provide me with your street address, city, state, and zip code so that we can locate your property and better understand your situation? If you don't feel comfortable providing that information just yet, no worries. We can move forward with general information and follow up later.
------------------------------------
*User*: Sure my name is George Schulz. My email is georgeschulz33@gmail.com, phone is 571-431-9531, and